In [16]:
#!pip install langchain faiss-cpu langchain_core langchain_openai langchain_community >/dev/null

In [41]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
from langchain.indexes import SQLRecordManager, index
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

from langchain_community.docstore.in_memory import InMemoryDocstore
from faiss import IndexFlatL2

In [18]:
from google.colab import userdata
key = userdata.get('OpenAI') # здесь можно заменить название секрета колаба на свое (для ключа ОпенЭйАй)
os.environ["OPENAI_API_KEY"] = key

In [35]:
doc1 = Document(page_content="kitty", metadata={"source": "kitty.txt"})
doc2 = Document(page_content="doggy", metadata={"source": "doggy.txt"})

In [36]:
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents([doc1], embedding)

In [37]:
# Создаем Менеджера записей
collection_name = "test_index"
namespace = f"faiss/{collection_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)
record_manager.create_schema()

In [39]:
# Using the 'full' cleanup mode for this example
index([doc1,doc2], record_manager, vectorstore, cleanup="full", source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [40]:
doc1 = Document(page_content="kitty kitty", metadata={"source": "kitty.txt"})
index([doc1, doc2], record_manager, vectorstore_0, cleanup="full", source_id_key="source")

{'num_added': 1, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 1}

In [49]:
vectorstore.docstore._dict

{'bed90466-8e65-483f-ace0-711e80257263': Document(page_content='kitty', metadata={'source': 'kitty.txt'}),
 '2a7c9f28-7763-52c1-b3ea-9aef92133476': Document(page_content='kitty', metadata={'source': 'kitty.txt'}),
 '39997b8e-1d4a-55e7-91d4-a00189e27bfa': Document(page_content='doggy', metadata={'source': 'doggy.txt'})}

# Пустое хранилище

In [45]:
vectorstore_0 = FAISS(
            embedding_function=embedding,
            docstore=InMemoryDocstore(),
            index=IndexFlatL2(1536),
            index_to_docstore_id={},
        )

In [46]:
# Создаем Менеджера записей
collection_name = "new_index"
namespace = f"faiss/{collection_name}"
record_manager_0 = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)
record_manager.create_schema()

In [47]:
# Using the 'full' cleanup mode for this example
index([doc1,doc2], record_manager_0, vectorstore_0, cleanup="full", source_id_key="source")

{'num_added': 2, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [48]:
vectorstore_0.docstore._dict

{'18662629-78e4-5160-8022-4fed9e64fe3f': Document(page_content='kitty kitty', metadata={'source': 'kitty.txt'}),
 '39997b8e-1d4a-55e7-91d4-a00189e27bfa': Document(page_content='doggy', metadata={'source': 'doggy.txt'})}